In [52]:
import pygad
import numpy as np
from keras.datasets import mnist
import random

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [53]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

# Flatten the images and convert labels to integers
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

# Convert labels to integers
y_train = np.array(y_train, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)

def create_custom_nn(num_layers, layer_neurons, input_shape, output_shape, epsi):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(784,)))  # Update input_shape to match flattened images
    for i in range(num_layers-1):
        model.add(Dense(layer_neurons[i], activation='relu'))
    model.add(Dense(output_shape, activation='softmax'))
    model.add(BatchNormalization(epsilon=epsi))

    return model


def test_model(chromosome):
    model = create_custom_nn(chromosome[0], chromosome[1], 784, 10, chromosome[2])
    custom_optimizer = Adam(learning_rate=chromosome[3])

    # Compile the model with the custom optimizer
    model.compile(optimizer=custom_optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test), verbose=0)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

    return test_accuracy


In [54]:
def initialize_population(sol_per_pop, num_genes, low_end_nb_neurons, high_end_nb_neurons, low_end_randomized, high_end_randomized):
    population = []
    for _ in range(sol_per_pop):
        chromosome = []
        nb_layers = random.randint(1, 2)
        chromosome.append(nb_layers)
        nb_on_layers=[]
        for _ in range(nb_layers):
            nb_on_layers.append(random.randint(low_end_nb_neurons, high_end_nb_neurons))
        chromosome.append(nb_on_layers)
        chromosome.append(random.uniform(low_end_randomized, high_end_randomized))
        chromosome.append(random.uniform(low_end_randomized, high_end_randomized))
        population.append(chromosome)
    return population


def select_parents(population, num_parents_mating):
    accuracies = [test_model(chromosome) for chromosome in population]

    # Combine individuals with their corresponding accuracy
    individuals_with_accuracies = list(zip(population, accuracies))

    # Sort individuals based on accuracy in descending order
    individuals_with_accuracies.sort(key=lambda x: x[1], reverse=True)

    # Select the top individuals based on num_parents_mating
    selected_parents = individuals_with_accuracies[:num_parents_mating]

    # Extract parents and their accuracies separately
    parents, parent_accuracies = zip(*selected_parents)
    parents = list(parents)

    return parents, parent_accuracies


def crossover(parents):
    offspring = []
    for i in range(sol_per_pop - num_parents_mating):
        parent1 = parents[0]
        parent2 = parents[1]
        child = [0, [], 0, 0]
        for j in range(len(parent1)):
            if j == 1:
                for k in range(child[0]):
                    if (parent1[0] > k) and (parent2[0] > k):
                        if random.random() < 0.5:
                            child[1].append(parent1[j][k])
                        else:
                            child[1].append(parent2[j][k])
                    elif parent1[0] > k:
                        child[1].append(parent1[j])
                    elif parent2[0] > k:
                        child[1].append(parent2[j])
                    else:
                        child[1].append(random.randint(low_end_nb_neurons, high_end_nb_neurons))
            else:
                if random.random() < 0.5:
                    child[j]=parent1[j]
                else:
                    child[j]=parent2[j]
        offspring.append(child)
    return offspring


def mutate(offspring):
    for i in range(len(offspring)):
        print(offspring[i])
        for j in range(len(offspring[i])):
            if j == 0:
                if random.random() < 0.1 and offspring[i][j] == 2:
                    x = random.randint(-1, 1)
                    offspring[i][j] = offspring[i][j] + x
                    if x == -1:
                        offspring[i][j+1] = offspring[i][j+1][:-1]
                    elif x == 1:
                        offspring[i][j+1].append(random.randint(low_end_nb_neurons, high_end_nb_neurons))
                elif random.random() < 0.1 and offspring[i][j] == 1:
                    x = random.randint(0, 1)
                    offspring[i][j] = offspring[i][j] + x
                    if x == 1:
                        offspring[i][j+1].append(random.randint(low_end_nb_neurons, high_end_nb_neurons))
                elif random.random() < 0.1 and offspring[i][j] == 3:
                    x = random.randint(-1, 0)
                    offspring[i][j] = offspring[i][j] + x
                    if x == -1:
                        offspring[i][j+1] = offspring[i][j+1][:-1]
            elif j == 1:
                if offspring[i][0] == 1:
                    if random.random() < 0.1 and 32 < offspring[i][j][0] < 132:
                        offspring[i][j] = offspring[i][j] + random.randint(-8, 8)
                else:
                    for k in range(offspring[i][0]):
                        if random.random() < 0.1 and 32 < offspring[i][j][k] < 132:
                            offspring[i][j][k] = offspring[i][j][k] + random.randint(-8, 8)
            else:
                if random.random() < 0.1 and 0.1<offspring[i][j]<0.9:
                    offspring[i][j] = offspring[i][j] + random.uniform(-0.1, 0.1)
    return offspring

def genetic_algorithm(num_generations, num_parents_mating, sol_per_pop, num_genes, low_end_nb_neurons, high_end_nb_neurons, low_end_randomized, high_end_randomized, aim_accuracy):
    population = initialize_population(sol_per_pop, num_genes, low_end_nb_neurons, high_end_nb_neurons, low_end_randomized, high_end_randomized)
    output_accuracy=0
    while output_accuracy < aim_accuracy: 
        parents, accuracies = select_parents(population, num_parents_mating)
        offspring = crossover(parents)
        offspring = mutate(offspring)
        population = parents + offspring
        best, output_accuracy = parents[0], accuracies[0] 
        print('Best Accuracy:', output_accuracy)
    
    return best, output
    

num_generations = 10000
num_parents_mating = 2
sol_per_pop = 10
num_genes = 3
low_end_nb_neurons = 32
high_end_nb_neurons = 128
low_end_randomized = 0
high_end_randomized = 1

aim_accuracy = 0.5


best, output = genetic_algorithm(num_generations, num_parents_mating, sol_per_pop, num_genes, low_end_nb_neurons, high_end_nb_neurons, low_end_randomized, high_end_randomized, aim_accuracy)
print(best, output)



Test Loss: 2.30259108543396 	Test Accuracy: 0.09799999743700027
Test Loss: 12.29907512664795 	Test Accuracy: 0.10090000182390213
Test Loss: 2.30259108543396 	Test Accuracy: 0.09769999980926514
Test Loss: 12.33084487915039 	Test Accuracy: 0.10090000182390213
Test Loss: 0.6225599050521851 	Test Accuracy: 0.8507000207901001


KeyboardInterrupt: 